# ZIP Codes

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import numpy as np
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from shapely.geometry import Point, MultiPoint
from geopandas.tools import sjoin
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
# pd.options.display.max_colwidth = None

---

## Scrape

#### Headers for request

In [3]:
gdf = gpd.read_file("data/zips_centroids.geojson")[
    [
        "ID",
        "NAME",
        "STATE_NAME",
        "ST_ABBREV",
        "TOTPOP_CY",
        "geometry",
    ]
].rename(
    columns={
        "ID": "zipcode",
        "NAME": "name",
        "STATE_NAME": "state_name",
        "ST_ABBREV": "state",
        "TOTPOP_CY": "pop",
    }
)

In [4]:
# Step 1: Ensure your GeoDataFrame is in a suitable CRS for distance calculations
# Reproject to a projection that accurately measures distances, such as Albers Equal Area for the US
gdf = gdf.to_crs(epsg=5070)  # EPSG:5070 is Albers Equal Area for the contiguous US
gdf = gdf[pd.notnull(gdf["geometry"])]

In [5]:
import geopandas as gpd
import numpy as np
from sklearn.cluster import DBSCAN

# Assuming 'gdf' is your original GeoDataFrame and it includes a 'state' column

# Exclude Hawaii ('HI') and Alaska ('AK') from the analysis
gdf_contiguous = gdf[~gdf["state"].isin(["HI", "AK"])]

# Ensure your GeoDataFrame is in a suitable CRS for accurate distance calculations (e.g., EPSG:5070 for the US)
gdf_contiguous = gdf_contiguous.to_crs(epsg=5070)

# Prepare coordinates for DBSCAN
coords = np.array(list(zip(gdf_contiguous.geometry.x, gdf_contiguous.geometry.y)))

# Run DBSCAN to cluster ZIP code centroids based on a 50-mile radius
db = DBSCAN(eps=80467, min_samples=1, metric="euclidean").fit(coords)

# Assign cluster labels to the GeoDataFrame
gdf_contiguous["cluster"] = db.labels_


# Define a function to select a representative point for each cluster
def get_representative_point(group):
    if len(group) > 1:
        # If there are multiple points, choose the point closest to the cluster's centroid for better representation
        cluster_centroid = group.unary_union.centroid
        closest_point = group.distance(cluster_centroid).idxmin()
        return group.loc[[closest_point]]
    return group.iloc[[0]]


# Select representative ZIP codes for each cluster
representative_zip_codes = (
    gdf_contiguous.groupby("cluster")
    .apply(get_representative_point)
    .reset_index(drop=True)
).to_crs(4326)

/var/folders/k0/l0l0k9f50w7cg_phqmhkhybm0000gx/T/ipykernel_33738/1015614764.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gdf_contiguous.groupby("cluster")


In [6]:
representative_zip_codes["longitude"] = representative_zip_codes.geometry.x
representative_zip_codes["latitude"] = representative_zip_codes.geometry.y

In [7]:
len(representative_zip_codes)

90

#### GeoJSON

In [8]:
representative_zip_codes.to_file(
    f"../_reference/representative_zip_codes_50_miles.geojson",
    driver="GeoJSON",
)

In [9]:
representative_zip_codes.drop(["geometry"], axis=1).to_json(
    f"../_reference/representative_zip_codes_50_miles.json", orient="records", indent=4
)